# RAG IAP 임베딩 생성 V7

원스토어 인앱결제 가이드 문서와 샘플 코드를 임베딩하여 FAISS 벡터 데이터베이스를 생성합니다.

In [16]:
import os
import re
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

## 1. 마크다운 문서 처리

In [17]:
source_file_name = "dev_center_guide_allmd.md"
source_file_path = "data/" + source_file_name

def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    """마크다운 문서를 계층적으로 분할합니다."""
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection"),
        ("#####", "subsubsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    chunk_idx = 0
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            chunk_idx += 1
            full_title = ""
            if "subsection" in metadata:
                full_title += f" > {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" > {metadata['subsubsection']}"
            if "subsubsubsection" in metadata:
                full_title += f" > {metadata['subsubsubsection']}"

            content = f"[{full_title}]\n\n{doc.page_content}"
            doc = Document(page_content=content, metadata={
                **doc.metadata,
                "type": "documentation",
                "source": source_file_name,
                "chunk_idx": chunk_idx
            })

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드 및 분할
str_md_file = load_markdown_file(source_file_path)
docs_markdown = hierarchical_markdown_split(str_md_file)

print(f"마크다운 문서 분할 완료: {len(docs_markdown)}개 청크")

마크다운 문서 분할 완료: 495개 청크


In [18]:
# docs_markdown에서 'launchPurchaseFlow()' 문자열을 포함하는 Documnet 를 출력

cnt = 0
for doc in docs_markdown:
    if 'PNS' in doc.page_content:
        # print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
        print(f"메타의 내용: {doc.metadata}")
        print(f"문서 내용: {doc.page_content}...")  # 처음 200자만 출력
        print("-" * 40)  # 구분선 
        cnt += 1
        
print(f"'PNS' 문자열을 포함하는 문서 개수: {cnt}")
        

메타의 내용: {'title': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'type': 'documentation', 'source': 'dev_center_guide_allmd.md', 'chunk_idx': 1}
문서 내용: []

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App Download

## 2. Kotlin 코드 처리

In [19]:
def extract_package(content: str) -> str:
    """Kotlin 파일에서 패키지 선언을 추출합니다."""
    package_match = re.search(r'package\s+([\w.]+)', content)
    return package_match.group(1) if package_match else ""

def extract_class_blocks(content: str) -> List[tuple]:
    """Kotlin 파일에서 클래스 블록을 추출합니다."""
    blocks = []
    
    # 클래스, 객체, 인터페이스, 함수 등 추출
    patterns = [
        (r'(class\s+\w+[\s\S]*?})', 'class'),
        (r'(object\s+\w+[\s\S]*?})', 'object'),
        (r'(interface\s+\w+[\s\S]*?})', 'interface'),
        (r'(fun\s+\w+[\s\S]*?})', 'function'),
        (r'(companion\s+object[\s\S]*?})', 'companion_object')
    ]
    
    for pattern, symbol_type in patterns:
        matches = re.finditer(pattern, content, re.MULTILINE)
        for match in matches:
            block_text = match.group(1)
            # 클래스/함수 이름 추출
            name_match = re.search(r'(class|object|interface|fun|companion\s+object)\s+(\w+)', block_text)
            symbol_name = name_match.group(2) if name_match else f"{symbol_type}_{len(blocks)}"
            
            blocks.append((f"{symbol_type}:{symbol_name}", block_text))
    
    return blocks

def load_kotlin_documents(project_root: str) -> list[Document]:
    """Kotlin 프로젝트에서 문서를 로드합니다."""
    project_root_path = Path(project_root)
    documents = []

    for file_path in project_root_path.rglob("*.kt"):
        rel_path = file_path.relative_to(project_root_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        package = extract_package(content)
        blocks = extract_class_blocks(content)
        
        for symbol, block in blocks:
            doc = Document(
                page_content=block,
                metadata={
                    "source": str(rel_path),
                    "symbol": symbol,
                    "package": package,
                    "type": "code",
                    "language": "kotlin"
                }
            )
            documents.append(doc)

    return documents

# Kotlin 코드 로드
docs_kotlin = load_kotlin_documents("code_sample/onestore_iap_release/")
print(f"Kotlin 코드 문서 로드 완료: {len(docs_kotlin)}개 청크")

# 샘플 출력
if docs_kotlin:
    print("\n첫 번째 코드 문서 샘플:")
    print(f"메타데이터: {docs_kotlin[0].metadata}")
    print(f"내용 (처음 200자): {docs_kotlin[0].page_content[:200]}...")

Kotlin 코드 문서 로드 완료: 218개 청크

첫 번째 코드 문서 샘플:
메타데이터: {'source': 'onestore_iap_sample/sample_subscription/src/main/java/com/onestore/sample/subscription/base/BaseRecyclerViewAdapter.kt', 'symbol': 'class:BaseRecyclerViewAdapter', 'package': 'com.onestore.sample.subscription.base', 'type': 'code', 'language': 'kotlin'}
내용 (처음 200자): class BaseRecyclerViewAdapter<T>: RecyclerView.Adapter<RecyclerView.ViewHolder>() {
    interface OnItemClickListener {
        fun <T> onItemClick(position: Int, item: T)
    }...


## 3. 문서 통합 및 임베딩 생성

In [20]:
# fixed_model_name = "deepseek-coder:6.7b"
fixed_model_name = "exaone3.5:latest"
# fixed_model_name = "mistral:latest"

def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=fixed_model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

# 모든 문서 통합
total_docs = docs_markdown + docs_kotlin
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")
print(f"- 마크다운 문서: {len(docs_markdown)}개")
print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 임베딩 생성 및 저장
output_dir = "models/faiss_vs_rag_iap_v2_06_" + fixed_model_name[:3]
os.makedirs(output_dir, exist_ok=True)
embed_and_save(total_docs, output_dir)

총 713개의 문서를 생성하였습니다.
- 마크다운 문서: 495개
- Kotlin 코드: 218개
✅ 임베딩 저장 완료: models/faiss_vs_rag_iap_v2_06_exa


## 4. 임베딩 검증

In [21]:
# 저장된 임베딩 로드 및 테스트
embedding_model = OllamaEmbeddings(model=fixed_model_name)
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

# 검색 테스트
test_queries = [
    # "인앱결제 초기화 방법",
    # "정기결제 구현 코드",
    "PNS",
    # "결제 검증 API"
]

for query in test_queries:
    print(f"\n🔍 검색 쿼리: {query}")
    results = loaded_db.similarity_search(query, k=20)
    
    for i, doc in enumerate(results, 1):
        print(f"\n결과 {i}:")
        print(f"타입: {doc.metadata.get('type', 'unknown')}")
        print(f"소스: {doc.metadata.get('source', 'unknown')}")
        print(f"내용: {doc.page_content}...")
        print("-" * 50)


🔍 검색 쿼리: PNS

결과 1:
타입: documentation
소스: dev_center_guide_allmd.md
내용: [ > 3.1 Conceptual View <a href="#id-greater-than-apiv7-greater-than-15.-greater-than-3" id="id-greater-than-apiv7-greater-than-15.-greater-than-3"></a>]

<figure><img src="https://1837360763-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2Fot0z57AnnXZ02C5qyePV%2Fuploads%2FO8FPHSpDj37xXZJyPd1X%2Fimage.png?alt=media&#x26;token=1b516c92-0a38-4187-9b38-a9d4cf5c07c4" alt=""><figcaption></figcaption></figure>...
--------------------------------------------------

결과 2:
타입: documentation
소스: dev_center_guide_allmd.md
내용: []

<table data-view="cards"><thead><tr><th></th><th></th><th></th></tr></thead><tbody><tr><td><strong>Flutter</strong></td><td>3.3.0</td><td></td></tr><tr><td><strong>Java SDK (Java 11)</strong></td><td><p>Purchase: v21.02.01</p><p>App License Checker: v2.2.1</p></td><td></td></tr><tr><td></td><td></td><td></td></tr></tbody></table>...
------------------------------------------------

## 5. 통계 정보

In [22]:
# 문서 타입별 통계
doc_types = {}
for doc in total_docs:
    doc_type = doc.metadata.get('type', 'unknown')
    doc_types[doc_type] = doc_types.get(doc_type, 0) + 1

print("📊 문서 타입별 통계:")
for doc_type, count in doc_types.items():
    print(f"- {doc_type}: {count}개")

# 코드 언어별 통계
code_languages = {}
for doc in total_docs:
    if doc.metadata.get('type') == 'code':
        lang = doc.metadata.get('language', 'unknown')
        code_languages[lang] = code_languages.get(lang, 0) + 1

if code_languages:
    print("\n💻 코드 언어별 통계:")
    for lang, count in code_languages.items():
        print(f"- {lang}: {count}개")

print(f"\n✅ 임베딩 생성 완료: {output_dir}")

📊 문서 타입별 통계:
- unknown: 1개
- documentation: 494개
- code: 218개

💻 코드 언어별 통계:
- kotlin: 218개

✅ 임베딩 생성 완료: models/faiss_vs_rag_iap_v2_06_exa


## 6. RAG 체인 생성 및 질문 응답

In [23]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# RAG 체인 구성 요소
# embedding_model = OllamaEmbeddings(model=fixed_model_name)
llm = ChatOllama(model=fixed_model_name, temperature=0.3)

# 검색기 설정
retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 20, "fetch_k": 30, "lambda_mult": 0.6}
)

# 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 단계별로 설명해주세요
4. 개발자 관점에서 실용적인 정보를 제공해주세요
5. 반드시 embedding된 문서(컨텍스트) 내용을 토대로 LLM 지식을 추가해주세요. 
6. 컨텍스트에 없는 내용은 "해당 정보를 찾을 수 없습니다"라고 답변하세요
7. 원스토어 이외의 정보는 답변하지 마세요.
8. url 등을 노출할때 onstore가 포함되지 않는 경우 노출되지 않아야 합니다.

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인이 생성되었습니다!")

✅ RAG 체인이 생성되었습니다!


## 7. 질문 응답 테스트

In [25]:
# 테스트 질문들
test_questions = [
    # "원스토어 인앱결제 초기화는 어떻게 하나요?",
    # "정기결제 구현 방법을 알려주세요",
    # "PNS(Payment Notification Service) 설정 방법은?",
    # "결제 검증은 어떻게 하나요?",
    # "관리형 상품과 구독형 상품의 차이점은?",
    # "원스토어의 PNS란 무엇이고 어떻게 구현하나요? 메시지 수신 서버의 관점에서 Java SpringFramework로 구현한 코드를 예제를 알려주세요",
    "원스토어의 PNS는 무엇이고 메세지 규격은 어떻게 됩니까?."
]

print("🤖 RAG 체인 테스트 시작\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

🤖 RAG 체인 테스트 시작


📝 질문 1: 원스토어의 PNS는 무엇이고 메세지 규격은 어떻게 됩니까?.
------------------------------------------------------------
원스토어의 **PNS (Push Notification Service)**는 개발자 서버로 결제 관련 알림 메시지를 전송하는 서비스입니다. 이를 통해 앱 내에서 결제 상태 변경이나 구독 관련 이벤트 등의 실시간 정보를 수신할 수 있습니다.

### PNS 메시지 규격에 대한 설명:

1. **설정 방법**:
   - 개발자 센터에서 `Apps > 상품 선택 > In-App 정보` 메뉴로 이동하여 `PNS 관리` 버튼을 클릭하여 PNS 수신 서버의 URL을 설정합니다.

2. **메시지 형식**:
   - PNS를 통해 전송되는 메시지는 일반적으로 JSON 형식을 따릅니다. 구체적인 메시지 구조는 다음과 같은 요소들을 포함할 수 있습니다:
     - **eventType**: 이벤트의 유형 (예: `Purchase`, `SubscriptionUpdate`, `SubscriptionCancelled` 등)
     - **payload**: 이벤트에 대한 추가 정보 (예: 구매 ID, 구독 상태 변경 정보 등)
     - **subscriptionId**: 정기 결제 구독 ID (정기 결제 관련 이벤트일 경우)
     - **purchaseId**: 구매 ID (일반 구매 이벤트일 경우)
     - **timestamp**: 이벤트 발생 시간

   **예시 메시지 구조**:
   ```json
   {
     "eventType": "Purchase",  // 이벤트 유형
     "payload": {
       "purchaseId": "1234567890",  // 구매 ID
       "subscriptionId": null,        // 정기 결제 구독 ID (해당 이벤트가 아닐 경우 null 또는 없음)
       " 

## 8. 대화형 질문 응답

In [ ]:
def ask_question(question: str) -> str:
    """질문에 대한 답변을 생성합니다."""
    try:
        answer = rag_chain.invoke(question)
        return answer
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

# 대화형 질문 예시
print("💬 대화형 질문 응답 테스트\n")

# 사용자 질문 입력 (실제 사용 시에는 input() 함수 사용)
sample_questions = [
    "인앱결제 SDK 버전은 어떻게 확인하나요?",
    "결제 실패 시 처리 방법을 알려주세요",
    "OAuth 인증 정보는 어디서 확인하나요?"
]

for question in sample_questions:
    print(f"\n❓ 질문: {question}")
    print("🔍 검색 중...")
    
    answer = ask_question(question)
    print(f"\n🤖 답변:\n{answer}")
    print("-" * 60)

## 9. 검색 결과 분석

In [ ]:
def analyze_search_results(question: str, top_k: int = 5):
    """검색 결과를 분석하여 상세 정보를 제공합니다."""
    print(f"🔍 질문: {question}")
    print(f"📊 상위 {top_k}개 검색 결과 분석\n")
    
    # 검색 결과 가져오기
    docs = retriever.get_relevant_documents(question)
    
    for i, doc in enumerate(docs[:top_k], 1):
        print(f"📄 결과 {i}:")
        print(f"   타입: {doc.metadata.get('type', 'unknown')}")
        print(f"   소스: {doc.metadata.get('source', 'unknown')}")
        if doc.metadata.get('type') == 'code':
            print(f"   심볼: {doc.metadata.get('symbol', 'unknown')}")
            print(f"   패키지: {doc.metadata.get('package', 'unknown')}")
        print(f"   내용: {doc.page_content[:150]}...")
        print("   " + "-" * 50)
    
    print(f"\n✅ 총 {len(docs)}개의 관련 문서를 찾았습니다.")

# 검색 결과 분석 테스트
analyze_search_results("인앱결제 초기화", top_k=3)
print("\n" + "=" * 80)
analyze_search_results("정기결제 구현", top_k=3)